# This is the Sinapse group's notebook - predictive model about PowerCo's churn.

This notebook contains:
- data exploration
- preprocessing
- modeling
- metrics
- analysis about the problem

# Installation of libraries

In [ ]:
# Instalação e atualização de bibliotecas: scikit-learn e klib
!pip install --upgrade scikit-learn
!pip install klib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 28.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
# Installation of libraries
import math
import pandas as pd
import numpy as np
import klib
import seaborn as sns
import matplotlib.pyplot as plt
from os import replace
from sklearn.linear_model import LinearRegression

# Importing tables from google drive

In [ ]:
#import from google drive
!gdown 1TLD79damXSqYHkUUMU754HUmKkYnK1jG

!gdown 1Nza3vUOL7WalKPG32BVQrso7J_Oeqj6x

!gdown 18B3pWhLVitH8H7Vaq88py19aaT2TqSeZ

Downloading...
From: https://drive.google.com/uc?id=1TLD79damXSqYHkUUMU754HUmKkYnK1jG
To: /content/base_clientes.csv
100% 5.25M/5.25M [00:00<00:00, 197MB/s]


# Defining the dataframes

In [ ]:
#important and defining the dataframes
data_customers = pd.read_csv('base_clientes.csv')

data_prices = pd.read_csv('base_precos.csv')

data_churns = pd.read_csv('base_hist_churn.csv', ',')

pd.set_option('display.max_columns', None)

# Functions

In [ ]:
# This function is used to handle date data, ensuring it follows standard conventions.
def to_dt(cell):
  try:
    if pd.isnull(cell):
        return cell
    elif int(cell.split('/')[0]) == 29 and int(cell.split('/')[1]) == 2:
        return cell.replace('29/02', '28/02')
    else:
        return cell
  except:
    return cell

In [ ]:
# Calculates consumption by dividing 'fix_price' by 'var_price' (avoiding division by zero), used for cost analysis.
def calculate_consumption(fix_price, var_price):
    if var_price != 0:
        return fix_price / var_price
    else:
        return 0

# Data Customers

In [ ]:
# Displays information about the 'data_customers' DataFrame, including data types, non-null counts, and memory usage
data_customers.info()

We will use our filled column to populate the *forecast_base_bill_ele* column via linear regression

In [ ]:
# Utilizes linear regression to impute missing values in 'forecast_base_bill_ele' and 'forecast_base_bill_year'.
regression_data = data_customers[['forecast_base_bill_ele', 'imp_cons']].dropna()

X = regression_data[['imp_cons']]
y = regression_data['forecast_base_bill_ele']

model_fill = LinearRegression()
model_fill.fit(X, y)

coef = model_fill.coef_[0]
intercept = model_fill.intercept_

data_customers['forecast_base_bill_ele'].fillna(data_customers['imp_cons'] * coef + intercept, inplace=True)
data_customers['forecast_base_bill_year'].fillna(data_customers['imp_cons'] * coef + intercept, inplace=True)

missing = data_customers[['forecast_base_bill_ele', 'forecast_base_bill_year']].isnull().sum()
missing

As the *has_gas* column is like t and f, just change this to 0 and 1 (a more interpretable way for the models)

In [ ]:
# Converts the 'has_gas' column values from 't' and 'f' to 1 and 0, respectively, and changes the data type to integer.
data_customers['has_gas'] = data_customers['has_gas'].map({'t': 1, 'f': 0})
data_customers['has_gas'] = data_customers['has_gas'].astype(int)

In the database, there are 3 columns defined by codes *channel_sales*, *origin_up*, *activity_new*:
For a correct interpretation of the models, these values must be converted to numbers.

In [ ]:
# Replaces NaN values in the 'channel_sales' column with 'no_fill', then creates dummy variables for 'channel_sales'.
data_customers['channel_sales'] = data_customers['channel_sales'].replace(np.nan, 'no_fill')
data_customers = pd.get_dummies(data_customers, columns=['channel_sales'])

In [ ]:
# Replaces NaN values in the 'origin_up' column with 'no_fill', then creates dummy variables for 'origin_up'.
data_customers['origin_up'] = data_customers['origin_up'].replace(np.nan, 'no_fill')
data_customers = pd.get_dummies(data_customers, columns=['origin_up'])

In [ ]:
# Replaces NaN values in the 'activity_new' column with 'no_fill' and converts it to binary values (1 for not NaN, 0 for NaN)
data_customers['activity_new'] = data_customers['activity_new'].replace(np.nan, 'no_fill')
data_customers['activity_new'] = data_customers['activity_new'].notna().astype(int)

You can see a column with all null values, which gives you the option to discard

In [ ]:
# Drops the 'campaign_disc_ele' column from the 'data_customers' DataFrame
data_customers.drop(['campaign_disc_ele'], axis=1, inplace=True)

Após uma análise das colunas importantes, definimos quais são as colunas que queremos no final do tratamento, por isso, retiramos as colunas : *forecast_base_bill_year*, *forecast_bill_12m*, *forecast_cons* e *date_first_activ*

In [ ]:
# Drops multiple columns ('forecast_base_bill_year', 'forecast_bill_12m', 'forecast_cons', 'date_first_activ')
data_customers.drop(['forecast_base_bill_year', 'forecast_bill_12m', 'forecast_cons', 'date_first_activ'], axis=1, inplace=True)

The next step is to treat the dates in the table, a quick analysis allows you to discover that they have invalid dates, so it is necessary to treat these dates. The way to treat them can be either through cyclic functions or by putting the year to represent

In [ ]:
# Applies the 'to_dt' function to convert date columns to a consistent format and then converts them to datetime objects.
data_customers['date_end'] = data_customers['date_end'].apply(to_dt)
data_customers['date_end'] = pd.to_datetime(data_customers['date_end'])

data_customers['date_activ'] = data_customers['date_activ'].apply(to_dt)
data_customers['date_activ'] = pd.to_datetime(data_customers['date_activ'])

data_customers['date_modif_prod'] = data_customers['date_modif_prod'].apply(to_dt)
data_customers['date_modif_prod'] = pd.to_datetime(data_customers['date_modif_prod'])

data_customers['date_renewal'] = data_customers['date_renewal'].apply(to_dt)
data_customers['date_renewal'] = pd.to_datetime(data_customers['date_renewal'])

In [ ]:
# Converts date columns to cyclical representations using sine and cosine transformations, dropping the original date columns.
data_customers['dia_do_ano'] = data_customers['date_end'].dt.dayofyear
data_customers['date_end_sin'] = np.sin(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers['date_end_cos'] = np.cos(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers.drop(['date_end', 'dia_do_ano'], axis=1, inplace=True)

data_customers['dia_do_ano'] = data_customers['date_activ'].dt.dayofyear
data_customers['date_activ_sin'] = np.sin(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers['dia_activ_cos'] = np.cos(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers.drop(['date_activ', 'dia_do_ano'], axis=1, inplace=True)

data_customers['dia_do_ano'] = data_customers['date_modif_prod'].dt.dayofyear
data_customers['date_modif_prod_sin'] = np.sin(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers['date_modif_prod_cos'] = np.cos(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers.drop(['date_modif_prod', 'dia_do_ano'], axis=1, inplace=True)

data_customers['dia_do_ano'] = data_customers['date_renewal'].dt.dayofyear
data_customers['date_renewal_sin'] = np.sin(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers['date_renewal_cos'] = np.cos(2 * np.pi * data_customers['dia_do_ano']/365)
data_customers.drop(['date_renewal', 'dia_do_ano'], axis=1, inplace=True)

Columns that have negative values were identified, for the model, they do not make sense, so we will remove these negative values from the table

In [ ]:
# Filters 'data_customers' DataFrame to retain rows where all specified columns in 'negatives' have non-negative values.
negatives = [
    'cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m',
    'forecast_cons_year', 'forecast_base_bill_ele','forecast_meter_rent_12m', 'forecast_price_pow_p1',
    'imp_cons', 'net_margin'
]

data_customers = data_customers[(data_customers[negatives] >= 0).all(axis=1)].reset_index(drop=True)

To ensure that all values in the *margin_net_pow_ele*, *margin_gross_pow_ele* columns are treated as positive values

In [ ]:
# Takes the absolute values of 'margin_net_pow_ele' and 'margin_gross_pow_ele' columns
data_customers['margin_net_pow_ele'] = data_customers['margin_net_pow_ele'].abs()
data_customers['margin_gross_pow_ele'] = data_customers['margin_gross_pow_ele'].abs()

To ensure that the database is as clean as possible to continue the feature engineering process, we must perform a cleaning and memory reduction process

In [ ]:
# Performs data cleaning on the 'data_customers' DataFrame using the klib library's data_cleaning function.
data_customers = klib.data_cleaning(data_customers)

# Data Prices

In [ ]:
# Displays information about the 'data_prices' DataFrame, including data types, non-null counts, and memory usage.
data_prices.info()

First, we can remove the *price_date* column, because it will not be useful for interpreting the dataset

In [ ]:
# Removes the 'price_date' column from 'data_prices' DataFrame and drops duplicate rows, keeping the first occurrence.
data_prices = data_prices.drop('price_date', axis=1).drop_duplicates()

It can be seen that in this table, there are 3 periods that can be treated as sets. This means we can make new columns that include the average of the other columns.

In [ ]:
# Calculates the mean of the columns 'price_p1_var', 'price_p2_var', and 'price_p3_var' and stores the result in 'price_var_mean'.
data_prices['price_var_mean'] = data_prices[['price_p1_var', 'price_p2_var', 'price_p3_var']].mean(axis=1)

In [ ]:
# Calculates the mean of the columns 'price_p1_fix', 'price_p2_fix', and 'price_p3_fix' and stores the result in 'price_fix_mean'.
data_prices['price_fix_mean'] = data_prices[['price_p1_fix', 'price_p2_fix', 'price_p3_fix']].mean(axis=1)

As the table has more than two hundred thousand rows, we have to group it in some way, in this case we will group by id and apply the median to describe the values

In [ ]:
# Groups 'data_prices' by 'id' and calculates the median for specified columns, resetting the index in the resulting DataFrame.
data_prices = data_prices.groupby('id').agg({
    'price_p1_var': 'median',
    'price_p2_var': 'median',
    'price_p3_var': 'median',
    'price_p1_fix': 'median',
    'price_p2_fix': 'median',
    'price_p3_fix': 'median',
    'price_var_mean': 'median',
    'price_fix_mean': 'median'
}).reset_index()

Now, let's create descriptive columns that bring a number for the total price per kWh and the price paid, in addition to the customer's consumption as well.

In [ ]:
# Computes 'price_var' and 'price_fix' as sums of corresponding price columns, calculates 'consumption', and drops original price columns.
data_prices['price_var'] = data_prices[['price_p1_var', 'price_p2_var', 'price_p3_var']].sum(axis=1)
data_prices['price_fix'] = data_prices[['price_p1_fix', 'price_p2_fix', 'price_p3_fix']].sum(axis=1)

data_prices['consumption'] = data_prices['price_fix'] / data_prices['price_var']

In [ ]:
# Resets the index of the 'data_prices' DataFrame, dropping the previous index and keeping it dropped.
data_prices.reset_index(drop=True)

Finally, let's clean this dataframe to make it lighter for the processes

In [ ]:
# Uses the 'klib' library's data cleaning function to perform data cleaning operations on 'data_prices' DataFrame.
data_prices = klib.data_cleaning(data_prices)

# Tables Merge

In [ ]:
# Performs an inner merge between 'data_customers' and 'data_prices' DataFrames based on the 'id' column, creating 'df_final'.
df_final = data_customers.merge(data_prices, how='inner', on='id')

In [ ]:
# Fills any remaining missing values in 'df_final' with zeros.
df_final = df_final.fillna(0)

In [ ]:
# Checks and counts the number of missing values in each column of the 'df_final' DataFrame.
df_final.isnull().sum()

In [ ]:
# Performs an inner merge between 'df_final' and 'data_churns' DataFrames based on the 'id' column, updating 'df_final'.
df_final = pd.merge(df_final, data_churns, on='id', how='inner')

In [ ]:
# Applying the data_cleaning function from the klib library to the df_final DataFrame
df_final = klib.data_cleaning(df_final)

In [ ]:
# Save the DataFrame df_final to a CSV file
df_final.to_csv('/content/df_final.csv')